In [1]:
import gym
import numpy
import scipy
env = gym.make('Taxi-v2')
new_state = env.reset()
from copy import copy

[2017-12-01 18:57:53,353] Making new env: Taxi-v2


In [2]:
observation = env.reset()
for i in range(100):
    #env.render()
    #print (observation)
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
    if done:
        print ("Done")
        break

In [3]:
q_values = numpy.random.rand(env.observation_space.n, env.action_space.n)
s_values = numpy.random.rand(env.observation_space.n)
gamma = 0.9
alpha = 0.5

In [5]:
def q_learn(iterations, q_values):
    total_reward = 0
    for i in range(iterations):
        new_state = env.reset()
        state = new_state 
        while True:
            action = numpy.argmax(q_values[state])
            new_state, reward, done, info = env.step(action)
            q_values[state][action] = reward + 0.5*numpy.max(q_values[new_state]) + 0.5*q_values[state][action]
            state = new_state
            total_reward += reward
            if done:
                break
    return q_values, total_reward

In [6]:
def dls(env_state, observation, curr_depth, max_depth):
    value_list = list()
    begin_state = copy(env)
    state = copy(env_state)
    if curr_depth == max_depth:
        return s_values[observation]
    else:
        for i in range(env.action_space.n):
            if curr_depth == 0:
                new_observation, reward, done, info = begin_state.step(i)
                state = copy(begin_state)
            else:
                new_observation, reward, done, info = state.step(i)
            if done:
                value = s_values[new_observation]
            else:
                value = reward + s_values[observation] + gamma * dls(state, new_observation, curr_depth+1, max_depth)
            value_list.append(value)
            state = copy(env)
        value = numpy.mean(value_list)
    return value

    

def dls_action(env_state, observation, max_depth):
    # Do depth Limited Search till max_depth on the mb_internal_environment
    value_list = numpy.zeros(env.action_space.n)
    for i in range(env.action_space.n):
        state = copy(env_state)
        new_observation, reward, done, info = state.step(i)
        value_list[i] = dls(state, new_observation, 0, max_depth)
    action = numpy.random.choice(numpy.flatnonzero(value_list == value_list.max()))
    return action

In [10]:
def dual(env, observation, max_depth, num_iters):
    j = 0
    reward_list = list()
    for i in range(num_iters):
        new_state = env.reset()  
        state = new_state
        sum_reward = 0
        count = 0
        while True:
            if j%(i+1) == 0:
                action = dls_action(env, observation, max_depth)
                new_state, reward, done, info = env.step(action)
                q_values[state][action] = reward + 0.5*numpy.max(q_values[new_state]) + 0.5*q_values[state][action]
                sum_reward += reward

                if done:
                    reward_list.append(sum_reward)
                    break
            else:
                action = numpy.argmax(q_values[state])
                new_state, reward, done, info = env.step(action)
                q_values[state][action] = reward + 0.5*numpy.max(q_values[new_state]) + 0.5*q_values[state][action]
                state = new_state
                sum_reward += reward
                if done:
                    reward_list.append(sum_reward)
                    break
            j += 1
    return reward_list
                

In [11]:
action = env.action_space.sample()
state = env.reset()

In [23]:
values = dual(env, state, max_depth=4, num_iters=100)

In [24]:
print (numpy.mean(values))

-58.32


In [ ]:
print (values[1])
